In [1]:
# =========================
# Library
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import seaborn as sns
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
from joblib import Parallel, delayed
import Levenshtein
import difflib
from contextlib import contextmanager
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import log_loss
import unicodedata
warnings.filterwarnings('ignore')
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# =========================
# Constant
# =========================
TRAIN_PATH = "../data/train.csv"
TARGET = "point_of_interest"

In [3]:
# =========================
# Settings
# =========================
fe = "065"
n_neighbors = 100
train_neighrbor_path = f"../output/fe/fe054/fe054_100_name_emb_near.pkl"

In [4]:
# =========================
# Functions
# =========================
def text_preprocess(text):
    text = str(text)
    text = text.replace(" ","")
    text = text.lower()
    text = unicodedata.normalize("NFKC",text)
    return text

def calc_distance_for_candidate(c1_array,c2_array):
    distance = np.zeros(len(c1_array),dtype=np.float32)
    for n,(c1,c2) in tqdm(enumerate(zip(c1_array,c2_array)),total=len(c1_array)):
        c1 = text_preprocess(c1)
        c2 = text_preprocess(c2)
        if (str(c1) != "nan") and (str(c2) != "nan"):
            distance[n] = np.array([Levenshtein.jaro_winkler(str(c1), str(c2))])
        else:
            distance[n] = np.nan
    return distance

In [5]:
# ============================
# Main
# ============================
train = pd.read_pickle(train_neighrbor_path)

In [6]:
train_raw = pd.read_csv(TRAIN_PATH)

In [10]:
train.drop(columns=["name","near_name"],inplace=True)
train = train.merge(train_raw[["id","name"]],on="id",how="left")
train_ = train_raw[["id","name"]].copy()
train_.columns = ["near_id","near_name"]
train = train.merge(train_,how="left",on="near_id")

In [13]:
columns = ['name']
for c in columns:
    distance = calc_distance_for_candidate(train[c].values, train[f"near_{c}"].values)
    distance = pd.DataFrame(distance)
    distance.columns = [f"{c}_jaro"]
    distance = reduce_mem_usage(distance)
    train = pd.concat([train,distance],axis=1)
    del distance
    gc.collect()

  0%|          | 0/112548740 [00:00<?, ?it/s]

Memory usage of dataframe is 429.34 MB
column =  1
0
Memory usage after optimization is: 429.34 MB
Decreased by 0.0%


In [14]:
train["distance"] = (train["latitude"] - train["near_latitude"])**2 + (train["longitude"] - train["near_longitude"])**2

In [15]:
train = reduce_mem_usage(train)
train.to_pickle(f"../output/fe/fe{fe}.pkl")

Memory usage of dataframe is 16958.91 MB
column =  25
0
Memory usage after optimization is: 16958.91 MB
Decreased by 0.0%
